# Домашнее задание 6

Это домашнее задание по материалам второго семинаров. Дедлайн по отправке - 23:55 24 марта. 

Домашнее задание выполняется в этом же Jupyter Notebook'e и присылается мне на почту: __beznosikov.an@phystech.edu__.

Решение каждой задачи необходимо поместить после её условия.

Файл должен называться: Фамилия_Имя_Optimization_HW_6

При полном запуске Вашего решения (Kernel -> Restart & Run All) все ячейки должны выполняться без ошибок. 

## Задача 1

Рассмотрим задачу минимизации:
\begin{equation}
\min_{x \in \mathbb{R}^d} f(x) = \frac{1}{2}x^T A x - b^T x,
\end{equation}
где в качестве матрицы $A$ - случайная матрица с $L =1000$, $\mu = 1$, $d = 100$, $b$ - так же случайный вектор.

__(а)__ Для такой задачи симулируем итерацию координатного спуска (смотри семинар). 

Для такого метода подберите шаг $\gamma$ для наилучшей сходимости постройте график сходимости (по оси $y$ - критерий $\| \nabla f(x^k)\|^2$, а по оси $x$ - число арифмитических операций при подсчете "градиента"). Постройте на этом же графике сходимость честного градиентного спуска c наилучшим шагом сходимости. Сделайте вывод о характере сходимости нового метода.

In [ ]:
import my_optimization as opt
import numpy as np

In [2]:
L = 1000
mu = 1
d = 100

args = {}
args['A'] = gen_A(d, mu, L)
args['b'] = np.random.random_sample(d)
x_0 = np.random.random_sample(d)

In [ ]:
'''
:CSGD: 
- gamma_th = 1/(4*L*d)
- gamma_exp

:GD:
- gamma = 1/L

:Method of calculating the number of iterations: 
- From the function f_quad_grad_f_j from opt 
we understand that the number of the number of arithmetic
operations is d*#times of calling f_quad_grad_f_j
'''

In [ ]:
def csgd_gamma_th(k, f, grad_f, x_k, x_true, args):
    return 1/(4 * L * d)

In [ ]:
csgd_gamma_th = opt.GradientOptimizer(f_quad, f_quad_grad, x_0, 
                                      criterium='||grad_f(x_k)||',
                                             gamma_k=gamma_coord_sgd_const,
                                             args=args, nabla_f_j=nabla_f_j,
                                             use_coord_sgd=True)
    _, iterations, errors, _ = coord_sgd.search()

In [ ]:
iterations = [i + 1 for i in range(n_iter)]
SGD_points = opt.Grad_Optimizer(n_iter, A, lr, b, x_0, optimizer = "SGD")
GD_points = opt.Grad_Optimizer(n_iter, A, lr, b, x_0, optimizer = "GD")
SGD_criteria_points = opt.criteria_points(A, b, SGD_points, criteria_type = "||grad_f||^2")
GD_criteria_points = opt.criteria_points(A, b, GD_points, criteria_type = "||grad_f||^2")
y = [SGD_criteria_points, GD_criteria_points]
label = ["SGD", "GD"]

opt.plot_graphs(iterations, y, label, "Comparison of SGD and GD", logscale = True, criteria_type = "||grad_f||^2")

__(б)__ В координатном спуске можно брать несколько координат вместо одной. Модифицируйте координнатный спуск, исходя из этого, пробуя разное число координат $b = 1, 5, 10, 20$, а также разные способы сэмплирования: все координатны независимо или зависимо, чтобы не было повторений в батче. Постройте графики аналогичные пункту (а). Сделайте вывод.

In [ ]:
#ответ

## Задача 2

Рассмотрим задачу линейной регрессии на датасете mushrooms с $\ell_2$-регуляризацией (коэффициент регуляризации равен $\frac{L}{1000}$). 

__(а)__ Оцените $\mu$ и $L$ для данной задачи. 

In [ ]:
#ответ

__(б)__ Реализуйте координатный SGD, SEGA для задачи линейно регрессии. Каким будете брать $b$? Как будете выбирать шаг и другие параметры методов?

In [ ]:
#ответ

__(в)__ Постройте графики сравнения всех методов: по оси $x$ используйте время или количество арифметических операций, по оси $y$ - $\|\nabla f(x^k)\|$ или точность на тесте. Таким образом, всего 4 графика. Сделайте сравнение и вывод.

In [ ]:
#ответ

__Бонусные пункты__

__(г)__ Попробуйте объеденить подходы SAGA/SVRG/L-SVRG и координатный SGD/SEGA (т.е. могут получиться пары SAGA + координатный, SVRG + SEGA и еще четыре других). Вам достаточно выбрать одну пару. Как при этом будет выглядеть итерация метода (запишите в явном виде)? Докажите сходимость данного подхода для $\mu$-сильно выпуклой $f$ и $L$-гладкой $f_i$.

In [ ]:
#ответ

__(д)__ Реализуйте новый метод. Сравните его с двумя методами, которые были взяты за базу, на задаче линейной регресии. Способы сравнения методов придумайте самостоятельно. Сделайте вывод.

In [ ]:
#ответ